Import the necessary packages

In [1]:
using Pkg
Pkg.activate("../libs/")
using Lux, Random, NNlib, Zygote, LuxCUDA, CUDA, FluxMPI, JLD2, DICOM
using Images
using ImageView
using MLUtils
using Optimisers, Statistics, imageToolBox

# BAC_root_dir = "/home/molloi-lab/Desktop/Project BAC/BAC_Full_Data"
# excel_path = "/home/molloi-lab/Desktop/Project BAC/BAC project/Ca mass calculation Final.xlsx"
# V_P_dict = Dict("LCC" => 1, "LMLO" => 2, "RCC" => 3, "RMLO" => 4)
# strs = ["LCC", "LMLO", "RCC", "RMLO"]

# image_dirs = ["Patientstudy_BAC_2013a/data_from_KP", 
# "Patientstudy_BAC_2013b/data_from_KP", 
# "Patientstudy_BAC_2014/data_from_KP", 
# "Patientstudy_BAC_2015a/data_from_KP", 
# "Patientstudy_BAC_2015b/data_from_KP", 
# "Patientstudy_BAC_2016/Data from KP"]

# save_dir = "/media/molloi-lab/System Files/BAC_dataset/Clean_Dataset_full"
# save_csv_dir = "/media/molloi-lab/System Files/BAC_dataset/splited_data.csv"
train_output_dir = "/media/molloi-lab/System Files/BAC_dataset/dataset_for_ML_full"

patch_size = 256
patch_size_half = round(Int, patch_size/2)

# SIDs = readdir(save_dir)
# num_SIDs = size(SIDs)[1];

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


┌ Warning: MPI Implementation is not CUDA Aware.
└ @ FluxMPI /home/molloi-lab/.julia/packages/FluxMPI/OM5f6/src/FluxMPI.jl:28


Gtk-Message: 18:24:15.824: Failed to load module "canberra-gtk-module"
Gtk-Message: 18:24:15.825: Failed to load module "canberra-gtk-module"


128

# 1. Prepare Data

## 1.1 Helper functions

In [2]:
"""
    This function zoom all pixel values into [0, 1].
"""
function zoom_pxiel_values(img)
    a, b = minimum(img), maximum(img)
    img_ = (img .- a) ./ (b - a)
    return img_
end

function normalize_img(img)
    m = maximum(img)
    img = m .- img
    a = mean(img)
    s = std(img)
    img = (img .- a) ./ s 
    # println("mean = $(mean(img)), std = $(std(img))")
    return img
end

"""
    This function takes in a img of various size, 
    returns patches with size = patch_size * patch_size.
"""
function patch_image(img, mask, lbl, num_patches_empty, num_patches_non_empty)
    # img_max = maximum(img)
    # img = img_max .- img
    img = normalize_img(img)
    s = size(img)
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
    img_patches_empty = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_empty)
    img_patches_non_empty  = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_non_empty)
    lbl_patches_non_empty  = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_non_empty)
    ct_empty , ct_non_empty = 0, 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # check patch
            if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
                # save patch
                if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                    ct_non_empty += 1
                    img_patches_non_empty[:, :, 1, ct_non_empty] = img[x_start:x_end, y_start:y_end]
                    lbl_patches_non_empty[:, :, 1, ct_non_empty] = lbl[x_start:x_end, y_start:y_end]
                else
                    ct_empty += 1
                    img_patches_empty[:, :, 1, ct_empty] = img[x_start:x_end, y_start:y_end]
                end
            end
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            # save patch
            if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                ct_non_empty += 1
                img_patches_non_empty[:, :, 1, ct_non_empty] = (img[x_start:x_end, y_start:y_end])
                lbl_patches_non_empty[:, :, 1, ct_non_empty] = lbl[x_start:x_end, y_start:y_end]
            else
                ct_empty += 1
                img_patches_empty[:, :, 1, ct_empty] = (img[x_start:x_end, y_start:y_end])
            end
        end
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            # save patch
            if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                ct_non_empty += 1
                img_patches_non_empty[:, :, 1, ct_non_empty] = (img[x_start:x_end, y_start:y_end])
                lbl_patches_non_empty[:, :, 1, ct_non_empty] = lbl[x_start:x_end, y_start:y_end]
            else
                ct_empty += 1
                img_patches_empty[:, :, 1, ct_empty] = (img[x_start:x_end, y_start:y_end])
            end
        end
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # check patch
    if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
        # save patch
        if sum(lbl[x_start:x_end, y_start:y_end]) > 0
            ct_non_empty += 1
            img_patches_non_empty[:, :, 1, ct_non_empty] = (img[x_start:x_end, y_start:y_end])
            lbl_patches_non_empty[:, :, 1, ct_non_empty] = lbl[x_start:x_end, y_start:y_end]
        else
            ct_empty += 1
            img_patches_empty[:, :, 1, ct_empty] = (img[x_start:x_end, y_start:y_end])
        end
    end
    # return
    return img_patches_empty, img_patches_non_empty, lbl_patches_non_empty
end

"""
    This function fixs the path to the images and labels.
"""
function fix_path!(data_set)
    num_data = size(data_set)[1]
    Threads.@threads for i = 1 : num_data
        for j = 1 : 2
            for k = 1 : 4
                # modify img path
                splited = split(deepcopy(data_set[i][j][k]), "\\")
                if size(splited)[1] > 1
                    new_path = joinpath("../collected_dataset_for_ML", joinpath(splited[4:end]))
                    data_set[i][j][k] = new_path
                end
            end
        end
    end
end

"""
    Helper functions for `get_num_of_imgs(data_set)`.
"""
function get_num_of_forgound_patches(img, lbl)
    s = size(img)
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
    ct_empty, ct_non_empty = 0, 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # check patch
            if mean(img[x_start:x_end, y_start:y_end]) > 0.35
                if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                    ct_non_empty += 1
                else
                    ct_empty += 1
                end
            end
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # check patch
        if mean(img[x_start:x_end, y_start:y_end]) > 0.35
            if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                ct_non_empty += 1
            else
                ct_empty += 1
            end
        end
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # check patch
        if mean(img[x_start:x_end, y_start:y_end]) > 0.35
            if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                ct_non_empty += 1
            else
                ct_empty += 1
            end
        end
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # check patch
    if mean(img[x_start:x_end, y_start:y_end]) > 0.35
        if sum(lbl[x_start:x_end, y_start:y_end]) > 0
            ct_non_empty += 1
        else
            ct_empty += 1
        end
    end
    return ct_empty, ct_non_empty
end

"""
    This function check how many number of images and labels there will be after patching.
"""
function get_num_of_imgs(data_set)
    num_data = size(data_set)[1]
    cts_empty = Array{Int}(undef, num_data*4)
    cts_non_empty = Array{Int}(undef, num_data*4)
    Threads.@threads for i = 1 : num_data
        @views t = train_set[i]
        for j = 1 : 4
            # read DICOM
            img = Float32.(read_dicom(t[1][j])[4])
            # read png mask images
            mask = get_breast_mask(img)
            # read label patching
            lbl = Images.load(t[2][j])
            # count non-background patches and save
            cts_empty[(i-1)*4+j], cts_non_empty[(i-1)*4+j] = get_num_of_forgound_patches(mask, lbl)
        end
    end
    return cts_empty, cts_non_empty
end

get_num_of_imgs

## 1.2 Prepare Data

In [3]:
data_dir1 = "/home/molloi-lab/Desktop/BAC Dataset/";
data_dir2 = joinpath(data_dir1, "collected_dataset_for_ML/train");

In [4]:
isdir(data_dir2)

true

In [5]:
@load "clean_set_step2_for_ubuntu.jld2" train_set valid_set

for i = 1: size(train_set)[1]
    for k = 1 : 2
        for j = 1: size(train_set[i][k])[1]
            splited = split(train_set[i][k][j], '/')
            p = data_dir2
            for w = 1 : size(splited)[1]
                if length(splited[w]) >= 5 && (splited[w][1:5] == "image" || splited[w][1:5] == "label")
                    p = joinpath(p, joinpath(splited[w:end]))
                    break
                end
            end
            train_set[i][k][j] = p
            if !isfile(p)
                println(i)
            end
        end
    end
end

### 1.2.1 Load train set & valid set
container format: patch_size * patch_size * 1 * num_imgs

In [6]:
# get num of total patches(train)
cts_empty, cts_non_empty = get_num_of_imgs(train_set)
total_empty, total_non_empty = sum(cts_empty), sum(cts_non_empty)
# num_patches_train_ = sum(ct_patches_train)

(269453, 61334)

In [7]:
total_empty

269453

In [8]:
total_non_empty

61334

In [10]:
GC.gc(true)
# runtime: 60s
num_train_data = size(train_set)[1]
train_container_images_empty = Array{Float16, 4}(undef, patch_size, patch_size, 1, total_empty)
train_container_images_non_empty = Array{Float16, 4}(undef, patch_size, patch_size, 1, total_non_empty)
train_container_masks_non_empty = Array{Float16, 4}(undef, patch_size, patch_size, 1, total_non_empty)

Threads.@threads for i = 1 : num_train_data
    start_idx_empty = sum(cts_empty[1:(i-1)*4])+1
    start_idx_non_empty = sum(cts_non_empty[1:(i-1)*4])+1
    for j = 1 : 4 # 4 images each patient
        num_patches_empty = cts_empty[(i-1)*4+j]
        num_patches_non_empty = cts_non_empty[(i-1)*4+j]
        # read dicom image
        img = dcm_parse(train_set[i][1][j])[(0x7fe0, 0x0010)]
        # read png mask
        # mask = Images.load(train_set[i][1][j][1:end-3] * "png") commemted out because some masks are not correct
        mask = deepcopy(Float32.(img))
        mask = 1f0 .- round.(zoom_pxiel_values(mask))
        # read png image
        lbl = Images.load(train_set[i][2][j])
        # process image
        img_patches_empty, img_patches_non_empty, lbl_patches_non_empty = patch_image(img, mask, lbl, num_patches_empty, num_patches_non_empty)
        # set end idx 
        end_idx_empty = start_idx_empty+num_patches_empty-1
        end_idx_non_empty = start_idx_non_empty+num_patches_non_empty-1
        # save
        train_container_images_empty[:, :, 1, start_idx_empty : end_idx_empty] = img_patches_empty
        train_container_images_non_empty[:, :, 1, start_idx_non_empty : end_idx_non_empty] = img_patches_non_empty
        train_container_masks_non_empty[:, :, 1, start_idx_non_empty : end_idx_non_empty] = lbl_patches_non_empty
        # set idx
        start_idx_empty = end_idx_empty
        start_idx_non_empty = end_idx_non_empty
    end
end
# 1. Empty : non-empty ~= 1:1
# cut size to 20%
@info "Cutting size to 20%..."
# reduced_size = div(size(train_container_images_empty, 4), 5)
reduced_size = total_non_empty
random_indices = randperm(size(train_container_images_empty, 4))
selected_indices = random_indices[1:reduced_size]
train_container_images_empty = train_container_images_empty[:, :, :, selected_indices]
train_container_masks_empty = zeros(Float32, size(train_container_images_empty))
# combine 
@info "Merging..."
total_size = total_non_empty + reduced_size
train_container_images = Array{Float16, 4}(undef, patch_size, patch_size, 1, total_size)
train_container_images[:, :, :, 1:total_non_empty] .= train_container_images_non_empty
train_container_images[:, :, :, total_non_empty + 1:end] .= train_container_images_empty
train_container_masks = Array{Float16, 4}(undef, patch_size, patch_size, 1, total_size)
train_container_masks[:, :, :, 1:total_non_empty] .= train_container_masks_non_empty
train_container_masks[:, :, :, total_non_empty + 1:end] .= train_container_masks_empty
# shuffle
@info "Shuffling..."
random_indices = randperm(size(train_container_images, 4))
train_container_images = train_container_images[:, :, :, random_indices]
train_container_masks = train_container_masks[:, :, :, random_indices]
# summary
@info "num_empty = $(size(train_container_images_empty)[4]), num_non_empty = $(size(train_container_images_non_empty)[4])"

# # 2. Only non-empty
# # combine 
# train_container_images = train_container_images_non_empty
# train_container_masks = train_container_masks_non_empty
# # summary
# @info "num_non_empty = $(size(train_container_images_non_empty[4]))"


GC.gc(true)

┌ Info: Cutting size to 20%...
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/Legacy code/4.1_Create_data_loader.ipynb:38
┌ Info: Merging...
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/Legacy code/4.1_Create_data_loader.ipynb:46


┌ Info: Shuffling...
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/Legacy code/4.1_Create_data_loader.ipynb:55
┌ Info: num_empty = 61334, num_non_empty = 61334
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/Legacy code/4.1_Create_data_loader.ipynb:60


In [11]:
# # get num of total patches(valid)
# ct_patches_valid = get_num_of_imgs(valid_set)
# num_patches_valid = sum(ct_patches_valid)
# GC.gc(true)

In [12]:
# # runtime: 7.5s
# num_valid_data = size(valid_set)[1]
# valid_container_images = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_valid)
# valid_container_masks = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_valid)
# Threads.@threads for i = 1 : num_valid_data
#     start_idx = sum(ct_patches_valid[1:(i-1)*4])+1
#     for j = 1 : 4 # 4 images each patient
#         num_patches = ct_patches_valid[(i-1)*4+j]
#         # read dicom image
#         img = dcm_parse(valid_set[i][1][j])[(0x7fe0, 0x0010)]
#         # read png mask
#         mask = Images.load(valid_set[i][1][j][1:end-3] * "png")
#         # read png image
#         lbl = Images.load(valid_set[i][2][j])
#         # process image
#         img_patches, lbl_patches = patch_image(img, mask, lbl, num_patches)
#         # set end idx 
#         end_idx = start_idx+num_patches-1
#         # save
#         valid_container_images[:, :, 1, start_idx : end_idx] = img_patches
#         valid_container_masks[:, :, 1, start_idx : end_idx] = lbl_patches
#         start_idx = end_idx
#     end
# end

### 1.2.2 Save data

In [13]:
total_num_patches = size(train_container_images)[4]

122668

In [14]:
each_gpu_num_patches = round(Int, total_num_patches/4)

30667

In [15]:
GC.gc(true)
b_s = 7

7

In [16]:
train_container_images = Float32.(train_container_images)
train_container_masks = Float32.(train_container_masks);

256×256×1×122668 Array{Float32, 4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱       ⋮                        ⋮
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0 

In [17]:
Threads.@threads for i = 1 : 4
    start_idx = 1 + each_gpu_num_patches*(i-1)
    end_idx = i==4 ? total_num_patches : each_gpu_num_patches*i
    train_loader = MLUtils.DataLoader((data=train_container_images[:,:,:,start_idx:end_idx], 
        label=train_container_masks[:,:,:,start_idx:end_idx]), batchsize=b_s)
    @save "../train_loader_$(i)_small.jld2" train_loader
    train_loader = nothing
end
GC.gc(true)

In [18]:
# @sync begin
#     @async begin
#         train_loader_1 = MLUtils.DataLoader((data=train_container_images[:,:,:,1:each_gpu_num_patches], 
#         label=train_container_masks[:,:,:,1:each_gpu_num_patches]), batchsize=b_s)
#         @save "train_loader_1.jld2" train_loader_1
#         train_loader_1 = nothing
#     end
#     @async begin
#         train_loader_2 = MLUtils.DataLoader((data=train_container_images[:,:,:,1+each_gpu_num_patches : each_gpu_num_patches*2], 
#         label=train_container_masks[:,:,:,1+each_gpu_num_patches : each_gpu_num_patches*2]), batchsize=b_s)
#         @save "train_loader_2.jld2" train_loader_2
#         train_loader_2 = nothing
#     end
#     @async begin
#         train_loader_3 = MLUtils.DataLoader((data=train_container_images[:,:,:,1+each_gpu_num_patches*2 : each_gpu_num_patches*3], 
#         label=train_container_masks[:,:,:,1+each_gpu_num_patches*2 : each_gpu_num_patches*3]), batchsize=b_s)
#         @save "train_loader_3.jld2" train_loader_3
#         train_loader_3 = nothing
#     end
#     @async begin
#         train_loader_4 = MLUtils.DataLoader((data=train_container_images[:,:,:,1+each_gpu_num_patches*3 : end], 
#         label=train_container_masks[:,:,:,1+each_gpu_num_patches*3 : end]), batchsize=b_s)
#         @save "train_loader_4.jld2" train_loader_4
#         train_loader_4 = nothing
#     end
# end
# GC.gc(true)

In [19]:
# sum(train_container_images[:,:,:,1+each_gpu_num_patches*3+15000])

In [20]:
# imshow(train_container_images[:,:,:,1+each_gpu_num_patches*3+15000])